In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests

In [2]:
#import csv data from 2020 to 2024
nba_data2023_2024 = pd.read_csv('nba2023-2024.csv', encoding='latin1', sep=';')
nba_data2022_2023 = pd.read_csv('nba2022-2023.csv', encoding='latin1', sep=';')
nba_data2021_2022 = pd.read_csv('nba2021-2022.csv', encoding='latin1', sep=';')
print(nba_data2023_2024.info())
print(nba_data2022_2023.info())
print(nba_data2021_2022.info())
#i have to deal with season 2020-2021 and season 2019-20 differently as data is in a different format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      735 non-null    int64  
 1   Player  735 non-null    object 
 2   Pos     735 non-null    object 
 3   Age     735 non-null    int64  
 4   Tm      735 non-null    object 
 5   G       735 non-null    int64  
 6   GS      735 non-null    int64  
 7   MP      735 non-null    float64
 8   FG      735 non-null    float64
 9   FGA     735 non-null    float64
 10  FG%     735 non-null    float64
 11  3P      735 non-null    float64
 12  3PA     735 non-null    float64
 13  3P%     735 non-null    float64
 14  2P      735 non-null    float64
 15  2PA     735 non-null    float64
 16  2P%     735 non-null    float64
 17  eFG%    735 non-null    float64
 18  FT      735 non-null    float64
 19  FTA     735 non-null    float64
 20  FT%     735 non-null    float64
 21  ORB     735 non-null    float64
 22  DR

In [3]:
#spliting columns and assigning the names as the original data didn't include headeers
nba_data2020_2021 = pd.read_csv('nba2020-2021.csv', encoding='latin1', sep=';', header=None)
nba_data2020_2021 = nba_data2020_2021[0].str.split(',', expand=True)
nba_data2020_2021.columns = [
    'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 
    '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 
    'BLK', 'TOV', 'PF', 'PTS'
]
#converting columns that contain numeric data into numeric format
numeric_columns = [
    'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 
    'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'
]
nba_data2020_2021[numeric_columns] = nba_data2020_2021[numeric_columns].apply(pd.to_numeric, errors='coerce')
nba_data2020_2021_cleaned = nba_data2020_2021.drop(index=0).reset_index(drop=True)
print(nba_data2020_2021_cleaned.head())
print(nba_data2020_2021_cleaned.info())

              Player Pos   Age   Tm     G    GS    MP   FG   FGA    FG%  ...  \
0   Precious Achiuwa  PF  21.0  MIA  28.0   2.0  14.6  2.6   4.4  0.590  ...   
1       Jaylen Adams  PG  24.0  MIL   6.0   0.0   2.8  0.2   1.3  0.125  ...   
2       Steven Adams   C  27.0  NOP  27.0  27.0  28.1  3.5   5.8  0.603  ...   
3        Bam Adebayo   C  23.0  MIA  26.0  26.0  33.6  7.4  12.9  0.573  ...   
4  LaMarcus Aldridge   C  35.0  SAS  18.0  18.0  26.7  5.9  12.5  0.476  ...   

     FT%  ORB  DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
0  0.561  1.3  2.7  4.0  0.6  0.4  0.5  1.0  1.9   6.5  
1  0.000  0.0  0.5  0.5  0.3  0.0  0.0  0.0  0.2   0.3  
2  0.468  4.3  4.6  8.9  2.1  1.0  0.6  1.7  1.9   8.0  
3  0.841  1.9  7.3  9.2  5.3  1.0  1.0  3.0  2.6  19.9  
4  0.762  0.8  3.5  4.3  1.9  0.4  0.9  0.9  1.5  14.1  

[5 rows x 29 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------ 

In [4]:
#data scrapping season 2019-2020
def scrape_team_per_game(team_initial):
    url = f"https://www.basketball-reference.com/teams/{team_initial}/2020.html#all_per_game-playoffs_per_game"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table = soup.find('table', {'id':'per_game'})
    headers = [th.text for th in table.find('thead').find_all('th')]
    rows = table.find('tbody').find_all('tr')
    data =[]
    for row in rows:
        cells = row.find_all('td')
        cells = [cell.text.strip() for cell in cells]
        data.append(cells)
    df = pd.DataFrame(data, columns=headers[1:])  #skipping the first header which is the rank column
    df['Team'] = team_initial  

    return df
#listing the initials of each team to put in the url
teams = [
    "MIL", "ATL", "BOS", "BRK", "CHO", "CHI", "CLE", "DAL", "DEN", "DET",
    "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIN", "NOP", "NYK",
    "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"
]
nba_data2019_2020 = pd.DataFrame()
for team in teams:
    team_data = scrape_team_per_game(team)
    if not team_data.empty:
        nba_data2019_2020 = pd.concat([nba_data2019_2020, team_data], ignore_index=True)
print(nba_data2019_2020.info())
print(nba_data2019_2020.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  591 non-null    object
 1   Age     591 non-null    object
 2   G       591 non-null    object
 3   GS      591 non-null    object
 4   MP      591 non-null    object
 5   FG      591 non-null    object
 6   FGA     591 non-null    object
 7   FG%     591 non-null    object
 8   3P      591 non-null    object
 9   3PA     591 non-null    object
 10  3P%     591 non-null    object
 11  2P      591 non-null    object
 12  2PA     591 non-null    object
 13  2P%     591 non-null    object
 14  eFG%    591 non-null    object
 15  FT      591 non-null    object
 16  FTA     591 non-null    object
 17  FT%     591 non-null    object
 18  ORB     591 non-null    object
 19  DRB     591 non-null    object
 20  TRB     591 non-null    object
 21  AST     591 non-null    object
 22  STL     591 non-null    ob

In [5]:
print(nba_data2019_2020.columns)

Index(['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Team'],
      dtype='object')


In [6]:
#changing columns that are numeric to numeric type
nba_data2019_2020.columns = [
    'Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', 
    '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 
    'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Team'
]
numeric_columns = [
    'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 
    'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'
]
nba_data2019_2020[numeric_columns] = nba_data2019_2020[numeric_columns].apply(pd.to_numeric, errors='coerce')
nba_data2019_2020_cleaned = nba_data2019_2020.drop(index=0).reset_index(drop=True)
print(nba_data2019_2020_cleaned.head())
print(nba_data2019_2020_cleaned.info())

             Player  Age   G  GS    MP   FG   FGA    FG%   3P  3PA  ...  ORB  \
0   Khris Middleton   28  62  59  29.9  7.6  15.3  0.497  2.4  5.7  ...  0.7   
1      Eric Bledsoe   30  61  61  27.0  5.5  11.5  0.475  1.2  3.5  ...  0.7   
2       Brook Lopez   31  68  67  26.7  4.3   9.9  0.435  1.5  4.8  ...  0.9   
3   Wesley Matthews   33  67  67  24.4  2.5   6.3  0.396  1.6  4.4  ...  0.3   
4  Donte DiVincenzo   23  66  24  23.0  3.5   7.7  0.455  1.3  3.7  ...  1.0   

   DRB  TRB  AST  STL  BLK  TOV   PF   PTS  Team  
0  5.5  6.2  4.3  0.9  0.1  2.2  2.3  20.9   MIL  
1  3.9  4.6  5.4  0.9  0.4  2.4  2.1  14.9   MIL  
2  3.7  4.6  1.5  0.7  2.4  1.0  2.4  12.0   MIL  
3  2.1  2.5  1.4  0.6  0.1  0.6  1.5   7.4   MIL  
4  3.8  4.8  2.3  1.3  0.3  1.4  1.7   9.2   MIL  

[5 rows x 28 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player

In [7]:
#looking for data that is missing values, to check why they are missing on the original webpage
missing_data_rows = nba_data2019_2020_cleaned[
    nba_data2019_2020_cleaned['3P%'].isna() |
    nba_data2019_2020_cleaned['eFG%'].isna() |
    nba_data2019_2020_cleaned['FT%'].isna() |
    nba_data2019_2020_cleaned['FG%'].isna()
]

print(missing_data_rows)
#data is missing because they didn't attempt any FT (e.g. if FT is 0, FT% is empty)

                   Player  Age   G  GS    MP   FG  FGA    FG%   3P  3PA  ...  \
34       Chandler Parsons   31   5   0  10.8  1.0  3.6  0.278  0.8  2.8  ...   
35            Paul Watson   25   2   0   8.5  0.0  3.5  0.000  0.0  1.5  ...   
47        Robert Williams   22  29   1  13.4  2.2  3.0  0.727  0.0  0.0  ...   
53             Tacko Fall   24   7   0   4.7  1.6  2.0  0.786  0.0  0.0  ...   
62         DeAndre Jordan   31  56   6  22.0  3.4  5.1  0.666  0.0  0.0  ...   
66             Donta Hall   22   5   0  17.0  2.8  3.6  0.778  0.0  0.0  ...   
76         Jamal Crawford   39   1   0   6.0  2.0  4.0  0.500  1.0  2.0  ...   
77         Henry Ellenson   23   5   0   3.0  0.2  1.4  0.143  0.0  0.8  ...   
86        Bismack Biyombo   27  53  29  19.4  2.8  5.2  0.543  0.0  0.0  ...   
106        Daniel Gafford   21  43   7  14.2  2.2  3.1  0.701  0.0  0.0  ...   
125            Ante Žižić   23  22   0  10.0  1.9  3.3  0.569  0.0  0.0  ...   
127           Matt Mooney   23   4   0  

In [8]:
#filling empty data with 0, as this is the true value
nba_data2019_2020_cleaned['FT%'].fillna(0, inplace=True)
nba_data2019_2020_cleaned['3P%'].fillna(0, inplace=True)
nba_data2019_2020_cleaned['FG%'].fillna(0, inplace=True)
nba_data2019_2020_cleaned['eFG%'].fillna(0, inplace=True)
nba_data2019_2020_cleaned['2P%'].fillna(0, inplace=True)
print(nba_data2019_2020_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  590 non-null    object 
 1   Age     590 non-null    int64  
 2   G       590 non-null    int64  
 3   GS      590 non-null    int64  
 4   MP      590 non-null    float64
 5   FG      590 non-null    float64
 6   FGA     590 non-null    float64
 7   FG%     590 non-null    float64
 8   3P      590 non-null    float64
 9   3PA     590 non-null    float64
 10  3P%     590 non-null    float64
 11  2P      590 non-null    float64
 12  2PA     590 non-null    float64
 13  2P%     590 non-null    float64
 14  eFG%    590 non-null    float64
 15  FT      590 non-null    float64
 16  FTA     590 non-null    float64
 17  FT%     590 non-null    float64
 18  ORB     590 non-null    float64
 19  DRB     590 non-null    float64
 20  TRB     590 non-null    float64
 21  AST     590 non-null    float64
 22  ST

In [9]:
#check data again before concatenating
print(nba_data2019_2020_cleaned.head()) #no rank column, only need to change team to tm, no Pos column
print(nba_data2019_2020_cleaned.info())
print(nba_data2020_2021_cleaned.head()) #no rank column
print(nba_data2020_2021_cleaned.info())
print(nba_data2021_2022.head()) #need to drop rank column
print(nba_data2021_2022.info())
print(nba_data2022_2023.head()) #need to drop rank column
print(nba_data2022_2023.info())
print(nba_data2023_2024.head()) #need to drop rank column
print(nba_data2023_2024.info())

             Player  Age   G  GS    MP   FG   FGA    FG%   3P  3PA  ...  ORB  \
0   Khris Middleton   28  62  59  29.9  7.6  15.3  0.497  2.4  5.7  ...  0.7   
1      Eric Bledsoe   30  61  61  27.0  5.5  11.5  0.475  1.2  3.5  ...  0.7   
2       Brook Lopez   31  68  67  26.7  4.3   9.9  0.435  1.5  4.8  ...  0.9   
3   Wesley Matthews   33  67  67  24.4  2.5   6.3  0.396  1.6  4.4  ...  0.3   
4  Donte DiVincenzo   23  66  24  23.0  3.5   7.7  0.455  1.3  3.7  ...  1.0   

   DRB  TRB  AST  STL  BLK  TOV   PF   PTS  Team  
0  5.5  6.2  4.3  0.9  0.1  2.2  2.3  20.9   MIL  
1  3.9  4.6  5.4  0.9  0.4  2.4  2.1  14.9   MIL  
2  3.7  4.6  1.5  0.7  2.4  1.0  2.4  12.0   MIL  
3  2.1  2.5  1.4  0.6  0.1  0.6  1.5   7.4   MIL  
4  3.8  4.8  2.3  1.3  0.3  1.4  1.7   9.2   MIL  

[5 rows x 28 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player

In [10]:
#drop rank column as it doesen't contribute anything for the project, basically the same as an index
def drop_rk_column(df):
    if 'Rk' in df.columns:
        df = df.drop(columns=['Rk'])
    return df
nba_data2021_2022 = drop_rk_column(nba_data2021_2022)
nba_data2022_2023 = drop_rk_column(nba_data2022_2023)
nba_data2023_2024 = drop_rk_column(nba_data2023_2024)
#changing Team column to Tm for later concatenation
def standardize_column_names(df):
    df = df.rename(columns={'Team': 'Tm'})
    return df
nba_data2019_2020_cleaned = standardize_column_names(nba_data2019_2020_cleaned)

In [11]:
# check for NaN in each years Pos column
def check_pos_nans(dataframe, year_label):
    if 'Pos' in dataframe.columns:
        nan_count = dataframe['Pos'].isna().sum()
    else:
        print(f"No 'Pos' column found in the {year_label} dataset.")
    
check_pos_nans(nba_data2019_2020_cleaned, '2019-2020')
check_pos_nans(nba_data2020_2021_cleaned, '2020-2021')
check_pos_nans(nba_data2021_2022, '2021-2022')
check_pos_nans(nba_data2022_2023, '2022-2023')
check_pos_nans(nba_data2023_2024, '2023-2024')

No 'Pos' column found in the 2019-2020 dataset.


In [12]:
nba_data2019_2020_cleaned['Season'] = '2019-2020'
nba_data2020_2021_cleaned['Season'] = '2020-2021'
nba_data2021_2022['Season'] = '2021-2022'
nba_data2022_2023['Season'] = '2022-2023'
nba_data2023_2024['Season'] = '2023-2024'

In [13]:
#combine datasets with the Pos column to create a mapping for the year 2019-2020
datasets = [nba_data2020_2021_cleaned, nba_data2021_2022, nba_data2022_2023, nba_data2023_2024]
combined_datasets = pd.concat(datasets)

In [14]:
#creating a map and filling positions in 2019-2020
position_mapping = combined_datasets.groupby('Player')['Pos'].agg(lambda x: x.mode().dropna().iloc[0] if not x.mode().empty else np.nan).dropna()
nba_data2019_2020_cleaned['Pos'] = nba_data2019_2020_cleaned['Player'].map(position_mapping)
#check how many positions are filled
filled_positions_count = nba_data2019_2020_cleaned['Pos'].notna().sum()
print(f"Filled positions in 2019-2020 dataset: {filled_positions_count}")

Filled positions in 2019-2020 dataset: 458


In [15]:
#change NaNs to Unknown and check if data is now complete for concatenation
nba_data2019_2020_cleaned['Pos'].fillna('Unknown', inplace=True)
print(nba_data2019_2020_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  590 non-null    object 
 1   Age     590 non-null    int64  
 2   G       590 non-null    int64  
 3   GS      590 non-null    int64  
 4   MP      590 non-null    float64
 5   FG      590 non-null    float64
 6   FGA     590 non-null    float64
 7   FG%     590 non-null    float64
 8   3P      590 non-null    float64
 9   3PA     590 non-null    float64
 10  3P%     590 non-null    float64
 11  2P      590 non-null    float64
 12  2PA     590 non-null    float64
 13  2P%     590 non-null    float64
 14  eFG%    590 non-null    float64
 15  FT      590 non-null    float64
 16  FTA     590 non-null    float64
 17  FT%     590 non-null    float64
 18  ORB     590 non-null    float64
 19  DRB     590 non-null    float64
 20  TRB     590 non-null    float64
 21  AST     590 non-null    float64
 22  ST

In [16]:
#concatenating all the data
nba_data_all_years = pd.concat([
    nba_data2019_2020_cleaned, 
    nba_data2020_2021_cleaned, 
    nba_data2021_2022, 
    nba_data2022_2023, 
    nba_data2023_2024
])
print(nba_data_all_years.head()) 
print(nba_data_all_years.info())

             Player   Age     G    GS    MP   FG   FGA    FG%   3P  3PA  ...  \
0   Khris Middleton  28.0  62.0  59.0  29.9  7.6  15.3  0.497  2.4  5.7  ...   
1      Eric Bledsoe  30.0  61.0  61.0  27.0  5.5  11.5  0.475  1.2  3.5  ...   
2       Brook Lopez  31.0  68.0  67.0  26.7  4.3   9.9  0.435  1.5  4.8  ...   
3   Wesley Matthews  33.0  67.0  67.0  24.4  2.5   6.3  0.396  1.6  4.4  ...   
4  Donte DiVincenzo  23.0  66.0  24.0  23.0  3.5   7.7  0.455  1.3  3.7  ...   

   TRB  AST  STL  BLK  TOV   PF   PTS   Tm     Season  Pos  
0  6.2  4.3  0.9  0.1  2.2  2.3  20.9  MIL  2019-2020   SF  
1  4.6  5.4  0.9  0.4  2.4  2.1  14.9  MIL  2019-2020   PG  
2  4.6  1.5  0.7  2.4  1.0  2.4  12.0  MIL  2019-2020    C  
3  2.5  1.4  0.6  0.1  0.6  1.5   7.4  MIL  2019-2020   SG  
4  4.8  2.3  1.3  0.3  1.4  1.7   9.2  MIL  2019-2020   SG  

[5 rows x 30 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 2851 entries, 0 to 734
Data columns (total 30 columns):
 #   Column  Non-Null Count  

In [17]:
#filter players who appear in all 5 seasons
players_in_all_seasons = nba_data_all_years.groupby('Player')['Season'].nunique()

#keep only players who have appeared in exactly 5 unique seasons
players_in_all_seasons = players_in_all_seasons[players_in_all_seasons == 5].index

#filter the original DataFrame to keep only these players
nba_filtered = nba_data_all_years[nba_data_all_years['Player'].isin(players_in_all_seasons)]

#display the filtered DataFrame
nba_filtered.head()
nba_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 786 entries, 0 to 734
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  786 non-null    object 
 1   Age     786 non-null    float64
 2   G       786 non-null    float64
 3   GS      786 non-null    float64
 4   MP      786 non-null    float64
 5   FG      786 non-null    float64
 6   FGA     786 non-null    float64
 7   FG%     786 non-null    float64
 8   3P      786 non-null    float64
 9   3PA     786 non-null    float64
 10  3P%     786 non-null    float64
 11  2P      786 non-null    float64
 12  2PA     786 non-null    float64
 13  2P%     786 non-null    float64
 14  eFG%    786 non-null    float64
 15  FT      786 non-null    float64
 16  FTA     786 non-null    float64
 17  FT%     786 non-null    float64
 18  ORB     786 non-null    float64
 19  DRB     786 non-null    float64
 20  TRB     786 non-null    float64
 21  AST     786 non-null    float64
 22  STL    

In [18]:
#export the data for EDA
nba_filtered.to_excel('EDA project1.xlsx', index=False)

In [19]:
#group by Player and Season, and calculate the mean of the relevant stats
df_grouped = nba_filtered.groupby(['Player', 'Season']).agg({
    'Age': 'first',  #age will remain the same for a season
    'G': 'mean',     #mean of games played
    'GS': 'mean',    #mean of games started
    'MP': 'mean',    #mean of minutes played
    'FG': 'mean',
    'FGA': 'mean',
    'FG%': 'mean',
    '3P': 'mean',
    '3PA': 'mean',
    '3P%': 'mean',
    '2P': 'mean',
    '2PA': 'mean',
    '2P%': 'mean',
    'eFG%': 'mean',
    'FT': 'mean',
    'FTA': 'mean',
    'FT%': 'mean',
    'ORB': 'mean',
    'DRB': 'mean',
    'TRB': 'mean',
    'AST': 'mean',
    'STL': 'mean',
    'BLK': 'mean',
    'TOV': 'mean',
    'PF': 'mean',
    'PTS': 'mean',
    'Tm': lambda x: ','.join(x.unique()),  #concatenate unique team initials
    'Pos': 'first'   #position will remain the same
}).reset_index()

#display the result
df_grouped.head()


,Player,Season,Age,G,GS,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm,Pos
0,Aaron Gordon,2019-2020,24.0,62.0,62.0,32.5,5.4,12.4,0.437,1.2,...,5.9,7.7,3.7,0.8,0.6,1.6,2.0,14.4,ORL,PF
1,Aaron Gordon,2020-2021,25.0,19.0,19.0,29.1,4.8,11.2,0.427,1.6,...,5.5,7.2,4.2,0.7,0.8,2.8,2.0,13.8,ORL,PF
2,Aaron Gordon,2021-2022,26.0,75.0,75.0,31.7,5.8,11.1,0.520,1.2,...,4.2,5.9,2.5,0.6,0.6,1.8,2.0,15.0,DEN,PF
3,Aaron Gordon,2022-2023,27.0,20.0,20.0,35.7,5.1,9.8,0.518,0.9,...,3.6,6.0,2.6,0.6,0.7,1.0,2.9,13.3,DEN,PF
4,Aaron Gordon,2023-2024,28.0,73.0,73.0,31.5,5.5,9.8,0.556,0.5,...,4.1,6.5,3.5,0.8,0.6,1.4,1.9,13.9,DEN,PF


In [20]:
#first, filter the DataFrame to include only the relevant seasons
filtered_df = df_grouped[df_grouped['Season'].isin(['2023-2024', '2022-2023', '2021-2022'])]

#create a copy of the DataFrame sorted by Player and Season to apply the shift function
sorted_df = df_grouped.sort_values(['Player', 'Season']).copy()

#define the columns that you want to apply the lead/lag to (excluding 'Player', 'Season', 'Tm', 'Pos')
columns_to_shift = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

#create shifted columns for the last year and the year before last
for col in columns_to_shift:
    # Shift by 1 for last year
    sorted_df[f'{col}_last_year'] = sorted_df.groupby('Player')[col].shift(1)
    # Shift by 2 for the year before last
    sorted_df[f'{col}_year_before_last'] = sorted_df.groupby('Player')[col].shift(2)

#now add the 'points_scored_next_year' from the 2023-2024 season
sorted_df['Points_scored_next_season'] = sorted_df.groupby('Player')['PTS'].shift(-1)

#filter the DataFrame again to include only seasons 2023-2024, 2022-2023, and 2021-2022
final_df = sorted_df[sorted_df['Season'].isin(['2023-2024', '2022-2023', '2021-2022'])]

#display the final DataFrame with the new columns
final_df.head(10)

,Player,Season,Age,G,GS,MP,FG,FGA,FG%,3P,...,STL_year_before_last,BLK_last_year,BLK_year_before_last,TOV_last_year,TOV_year_before_last,PF_last_year,PF_year_before_last,PTS_last_year,PTS_year_before_last,Points_scored_next_season
2,Aaron Gordon,2021-2022,26.0,75.0,75.0,31.700000,5.800000,11.1,0.520000,1.200000,...,0.80,0.800000,0.6,2.800000,1.600000,2.000000,2.0,13.800000,14.4,13.3
3,Aaron Gordon,2022-2023,27.0,20.0,20.0,35.700000,5.100000,9.8,0.518000,0.900000,...,0.70,0.600000,0.8,1.800000,2.800000,2.000000,2.0,15.000000,13.8,13.9
4,Aaron Gordon,2023-2024,28.0,73.0,73.0,31.500000,5.500000,9.8,0.556000,0.500000,...,0.60,0.700000,0.6,1.000000,1.800000,2.900000,2.0,13.300000,15.0,NaN
7,Aaron Holiday,2021-2022,25.0,42.0,10.0,16.233333,2.366667,5.4,0.441667,0.633333,...,0.80,0.100000,0.2,0.800000,1.300000,1.700000,1.8,7.400000,9.5,0.0
8,Aaron Holiday,2022-2023,26.0,1.0,0.0,4.000000,0.000000,0.0,0.000000,0.000000,...,0.50,0.100000,0.1,1.133333,0.800000,1.500000,1.7,6.400000,7.4,6.6
9,Aaron Holiday,2023-2024,27.0,78.0,1.0,16.300000,2.400000,5.3,0.446000,1.100000,...,0.70,0.000000,0.1,1.000000,1.133333,0.000000,1.5,0.000000,6.4,NaN
12,Al Horford,2021-2022,35.0,69.0,69.0,29.100000,3.900000,8.2,0.467000,1.300000,...,0.80,0.800000,0.9,1.300000,1.200000,1.700000,2.1,14.700000,11.9,6.7
13,Al Horford,2022-2023,36.0,20.0,20.0,30.900000,2.600000,6.6,0.386000,1.400000,...,0.90,1.300000,0.8,0.900000,1.300000,1.900000,1.7,10.200000,14.7,8.6
14,Al Horford,2023-2024,37.0,65.0,33.0,26.800000,3.300000,6.4,0.511000,1.700000,...,0.70,1.700000,1.3,0.600000,0.900000,2.400000,1.9,6.700000,10.2,NaN
17,Alex Len,2021-2022,28.0,39.0,10.0,15.900000,2.400000,4.5,0.534000,0.200000,...,0.35,1.066667,0.9,1.166667,1.050000,1.833333,2.3,5.433333,7.3,2.7


In [21]:
column_order = ['Player', 
                'Age', 'Age_last_year', 'Age_year_before_last',
                'G', 'G_last_year', 'G_year_before_last',
                'GS', 'GS_last_year', 'GS_year_before_last',
                'MP', 'MP_last_year', 'MP_year_before_last',
                'FG', 'FG_last_year', 'FG_year_before_last',
                'FGA', 'FGA_last_year', 'FGA_year_before_last',
                'FG%', 'FG%_last_year', 'FG%_year_before_last',
                '3P', '3P_last_year', '3P_year_before_last',
                '3PA', '3PA_last_year', '3PA_year_before_last',
                '3P%', '3P%_last_year', '3P%_year_before_last',
                '2P', '2P_last_year', '2P_year_before_last',
                '2PA', '2PA_last_year', '2PA_year_before_last',
                '2P%', '2P%_last_year', '2P%_year_before_last',
                'eFG%', 'eFG%_last_year', 'eFG%_year_before_last',
                'FT', 'FT_last_year', 'FT_year_before_last',
                'FTA', 'FTA_last_year', 'FTA_year_before_last',
                'FT%', 'FT%_last_year', 'FT%_year_before_last',
                'ORB', 'ORB_last_year', 'ORB_year_before_last',
                'DRB', 'DRB_last_year', 'DRB_year_before_last',
                'TRB', 'TRB_last_year', 'TRB_year_before_last',
                'AST', 'AST_last_year', 'AST_year_before_last',
                'STL', 'STL_last_year', 'STL_year_before_last',
                'BLK', 'BLK_last_year', 'BLK_year_before_last',
                'TOV', 'TOV_last_year', 'TOV_year_before_last',
                'PF', 'PF_last_year', 'PF_year_before_last',
                'PTS', 'PTS_last_year', 'PTS_year_before_last', 'Points_scored_next_season',
                'Tm', 'Season', 'Pos']
final_df = final_df[column_order]

final_df = final_df.round(2)

final_df = final_df[final_df['Season'] != '2023-2024']

In [22]:
positions = set([pos for sublist in final_df['Pos'].str.split('-') for pos in sublist])


for position in positions:
    final_df.loc[:, position] = final_df_without_2023['Pos'].apply(lambda x: 1 if position in x.split('-') else 0)

#display the first few rows of the modified dataframe
final_df.head()

/var/folders/9z/xd6qhps576s3xjk8023gqphw0000gn/T/ipykernel_55478/1254160001.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_without_2023.loc[:, position] = final_df_without_2023['Pos'].apply(lambda x: 1 if position in x.split('-') else 0)
/var/folders/9z/xd6qhps576s3xjk8023gqphw0000gn/T/ipykernel_55478/1254160001.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_without_2023.loc[:, position] = final_df_without_2023['Pos'].apply(lambda x: 1 if position in x.split('-') else 0)
/var/fol

,Player,Age,Age_last_year,Age_year_before_last,G,G_last_year,G_year_before_last,GS,GS_last_year,GS_year_before_last,...,PTS_last_year,PTS_year_before_last,Points_scored_next_season,Tm,Season,Pos,SG,SF,PG,C
2,Aaron Gordon,26.0,25.0,24.0,75.0,19.0,62.0,75.0,19.0,62.0,...,13.8,14.4,13.3,DEN,2021-2022,PF,0,0,0,0
3,Aaron Gordon,27.0,26.0,25.0,20.0,75.0,19.0,20.0,75.0,19.0,...,15.0,13.8,13.9,DEN,2022-2023,PF,0,0,0,0
7,Aaron Holiday,25.0,24.0,23.0,42.0,29.0,66.0,10.0,6.0,33.0,...,7.4,9.5,0.0,"TOT,WAS,PHO",2021-2022,PG,0,0,1,0
8,Aaron Holiday,26.0,25.0,24.0,1.0,42.0,29.0,0.0,10.0,6.0,...,6.4,7.4,6.6,ATL,2022-2023,PG,0,0,1,0
12,Al Horford,35.0,34.0,33.0,69.0,19.0,67.0,69.0,19.0,61.0,...,14.7,11.9,6.7,BOS,2021-2022,C,0,0,0,1


In [38]:
final_df.drop(columns=['Pos'], inplace=True)

cols = list(final_df.columns)

#move PF to the back of the df
cols.remove('PF')

cols.append('PF')

#reorder the dataframe columns
final_df = final_df[cols]

#display the first few rows of the modified dataframe
final_df.head()

,Player,Age,Age_last_year,Age_year_before_last,G,G_last_year,G_year_before_last,GS,GS_last_year,GS_year_before_last,...,PTS_last_year,PTS_year_before_last,Points_scored_next_season,Tm,Season,SG,SF,PG,C,PF
2,Aaron Gordon,26.0,25.0,24.0,75.0,19.0,62.0,75.0,19.0,62.0,...,13.8,14.4,13.3,DEN,2021-2022,0,0,0,0,1.0
3,Aaron Gordon,27.0,26.0,25.0,20.0,75.0,19.0,20.0,75.0,19.0,...,15.0,13.8,13.9,DEN,2022-2023,0,0,0,0,1.0
7,Aaron Holiday,25.0,24.0,23.0,42.0,29.0,66.0,10.0,6.0,33.0,...,7.4,9.5,0.0,"TOT,WAS,PHO",2021-2022,0,0,1,0,0.0
8,Aaron Holiday,26.0,25.0,24.0,1.0,42.0,29.0,0.0,10.0,6.0,...,6.4,7.4,6.6,ATL,2022-2023,0,0,1,0,0.0
12,Al Horford,35.0,34.0,33.0,69.0,19.0,67.0,69.0,19.0,61.0,...,14.7,11.9,6.7,BOS,2021-2022,0,0,0,1,0.0


In [40]:
#export the data for EDA
final_df.to_excel('DataWrangling2.0.xlsx', index=False)

In [39]:
print(final_df.columns)

Index(['Player', 'Age', 'Age_last_year', 'Age_year_before_last', 'G',
       'G_last_year', 'G_year_before_last', 'GS', 'GS_last_year',
       'GS_year_before_last', 'MP', 'MP_last_year', 'MP_year_before_last',
       'FG', 'FG_last_year', 'FG_year_before_last', 'FGA', 'FGA_last_year',
       'FGA_year_before_last', 'FG%', 'FG%_last_year', 'FG%_year_before_last',
       '3P', '3P_last_year', '3P_year_before_last', '3PA', '3PA_last_year',
       '3PA_year_before_last', '3P%', '3P%_last_year', '3P%_year_before_last',
       '2P', '2P_last_year', '2P_year_before_last', '2PA', '2PA_last_year',
       '2PA_year_before_last', '2P%', '2P%_last_year', '2P%_year_before_last',
       'eFG%', 'eFG%_last_year', 'eFG%_year_before_last', 'FT', 'FT_last_year',
       'FT_year_before_last', 'FTA', 'FTA_last_year', 'FTA_year_before_last',
       'FT%', 'FT%_last_year', 'FT%_year_before_last', 'ORB', 'ORB_last_year',
       'ORB_year_before_last', 'DRB', 'DRB_last_year', 'DRB_year_before_last',
       '